<a href="https://colab.research.google.com/github/Igprad01/Sentimen-Analisis/blob/main/Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANALISIS SENTIMEN CASE STUDY OPINI PUBLIK TWITTER

In [51]:
#install library
!pip install Sastrawi
!pip install nltk
!pip install -U scikit-learn

In [52]:
#import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk as nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
import csv
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


INPUT DATA SET

In [53]:
# load data
files = '/content/sample_data/Gabungan sample crawling JOT jkt48.csv'
df = pd.read_csv(files)
df.head()
# display(df)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1.873810e+18,Mon Dec 30 18:55:01 +0000 2024,0,Om JOT Liat deh Si up girls ngajak @RFritzy_JK...,1.873810e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/XJulianto/status/187380500346035...,1.818490e+18,XJulianto
1,1.873640e+18,Mon Dec 30 17:47:27 +0000 2024,0,@RFritzy_JKT48 Kalo oshinya JOT masih ga apa a...,1.873790e+18,https://pbs.twimg.com/media/GgEIPq1bYAExxgW.jpg,RFritzy_JKT48,in,"DKI Jakarta, Indonesia",0,0,0,https://x.com/Anggithaadr/status/1873788000431...,8.198810e+17,Anggithaadr
2,1.873760e+18,Mon Dec 30 15:52:30 +0000 2024,0,harapan untuk jkt48 yaitu membuat para jot sad...,1.873760e+18,NaN,NaN,in,NaN,0,0,0,https://x.com/blq_abilzz/status/18737590745775...,1.786020e+18,blq_abilzz
3,1.873630e+18,Mon Dec 30 15:25:31 +0000 2024,1,@_rybh @Maffi_f07 @Auwia_JKT48 Itu urusan inte...,1.873750e+18,NaN,_rybh,in,Salakanagara,0,1,0,https://x.com/gagitugie___/status/187375228321...,2.429397e+09,gagitugie___
4,1.873630e+18,Mon Dec 30 15:07:03 +0000 2024,0,@gagitugie___ @Maffi_f07 @Auwia_JKT48 Menunjuk...,1.873750e+18,NaN,gagitugie___,in,NaN,0,1,0,https://x.com/_rybh/status/1873747635750011333,3.181792e+09,_rybh


In [62]:
#case folding
lower_case = df['full_text'].str.lower()
display(lower_case)
#cleaning text
df_text = lower_case.str.replace(r'[^\w\s]', '')
df_text = lower_case.str.replace(r'\d+', '')
df_text = lower_case.str.replace(r'@','')
display(df_text)

,full_text
0,om jot liat deh si up girls ngajak @rfritzy_jk...
1,@rfritzy_jkt48 kalo oshinya jot masih ga apa a...
2,harapan untuk jkt48 yaitu membuat para jot sad...
3,@_rybh @maffi_f07 @auwia_jkt48 itu urusan inte...
4,@gagitugie___ @maffi_f07 @auwia_jkt48 menunjuk...
...,...
6211,lahkk makasih om jot see u @greesel_jkt48 http...
6212,@jambubiji08 @gracie_jkt48 harusnya faham jot om
6213,@greseella @greesel_jkt48 udah kena bedak jot ...
6214,@naim_pasha @officialjkt48 @y_chikajkt48 @f_fe...


,full_text
0,om jot liat deh si up girls ngajak rfritzy_jkt...
1,rfritzy_jkt48 kalo oshinya jot masih ga apa ap...
2,harapan untuk jkt48 yaitu membuat para jot sad...
3,_rybh maffi_f07 auwia_jkt48 itu urusan interna...
4,gagitugie___ maffi_f07 auwia_jkt48 menunjukan ...
...,...
6211,lahkk makasih om jot see u greesel_jkt48 https...
6212,jambubiji08 gracie_jkt48 harusnya faham jot om
6213,greseella greesel_jkt48 udah kena bedak jot aj...
6214,naim_pasha officialjkt48 y_chikajkt48 f_fenijk...


PRE-PROSESSING DATA

In [63]:
#tokenization #besok diperbaiki
nltk.download('punkt_tab')
token_text = df_text.apply(word_tokenize)
display(token)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,full_text
0,"[om, jot, liat, deh, si, up, girls, ngajak, rf..."
1,"[rfritzy_jkt48, kalo, oshinya, jot, masih, ga,..."
2,"[harapan, untuk, jkt48, yaitu, membuat, para, ..."
3,"[_rybh, maffi_f07, auwia_jkt48, itu, urusan, i..."
4,"[gagitugie___, maffi_f07, auwia_jkt48, menunju..."
...,...
6211,"[lahkk, makasih, om, jot, see, u, greesel_jkt4..."
6212,"[jambubiji08, gracie_jkt48, harusnya, faham, j..."
6213,"[greseella, greesel_jkt48, udah, kena, bedak, ..."
6214,"[naim_pasha, officialjkt48, y_chikajkt48, f_fe..."


In [43]:
#stopwords removal
stop_words = set(stopwords.words('indonesian'))
filtered_tokens = token_text.apply(lambda x: [word for word in x if word not in stop_words])
display(filtered_tokens)

,full_text
0,"[Om, JOT, Liat, deh, Si, up, girls, ngajak, RF..."
1,"[RFritzy_JKT, Kalo, oshinya, JOT, ga, nder, ?,..."
2,"[harapan, jkt, jot, sadar, TANPA, FANS, MEREKA..."
3,"[_rybh, Maffi_f, Auwia_JKT, Itu, urusan, inter..."
4,"[gagitugie___, Maffi_f, Auwia_JKT, Menunjukan,..."
...,...
6211,"[Lahkk, makasih, om, jot, See, u, Greesel_JKT]"
6212,"[JambuBiji, Gracie_JKT, Harusnya, faham, JOT, om]"
6213,"[greseella, Greesel_JKT, Udah, kena, bedak, JO..."
6214,"[naim_pasha, officialJKT, Y_ChikaJKT, F_FeniJK..."


In [44]:
#stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stemmed_tokens = filtered_tokens.apply(lambda x: [stemmer.stem(word) for word in x])
display(stemmed_tokens)

,full_text
0,"[om, jot, liat, deh, si, up, girls, ngajak, rf..."
1,"[rfritzy jkt, kalo, oshinya, jot, ga, nder, , ..."
2,"[harap, jkt, jot, sadar, tanpa, fans, mereka, ..."
3,"[rybh, maffi f, auwia jkt, itu, urus, internal..."
4,"[gagitugie, maffi f, auwia jkt, tunjuk, kalo, ..."
...,...
6211,"[lahkk, makasih, om, jot, see, u, greesel jkt]"
6212,"[jambubiji, gracie jkt, harus, faham, jot, om]"
6213,"[greseella, greesel jkt, udah, kena, bedak, jo..."
6214,"[naim pasha, officialjkt, y chikajkt, f fenijk..."


In [45]:
#download csv
token.to_csv('hasil_sementara.csv', index=False)

PELABELAN DATA

In [ ]:
# pelabelan data


In [ ]:
# modelisasi

In [ ]:
# extrasi fitur

In [ ]:
# klasifikasi sentimen

In [ ]:
# evaluasi hasil


In [ ]:
# visualisasi dan hasilnya